In [1]:
import os
import gym 
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
environment_name = "CartPole-v0"

In [3]:
env = gym.make(environment_name, render_mode="human")

c:\Users\Aditya Bankar\anaconda3\lib\site-packages\gym\envs\registration.py:555: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.warn(


In [4]:
episodes = 5
for episode in range(1, episodes + 1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()  # Render the environment without specifying a render mode
        action = env.action_space.sample()
        step_result = env.step(action)
        n_state, reward, done, info = step_result[:4]
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

Episode:1 Score:31.0
Episode:2 Score:17.0
Episode:3 Score:14.0
Episode:4 Score:41.0
Episode:5 Score:20.0


In [5]:
log_path = os.path.join('Training','Logs')

In [6]:
log_path

'Training\\Logs'

In [7]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda:env]) #Environment creation function
model = PPO('MlpPolicy',env,verbose = 1)


Using cpu device


c:\Users\Aditya Bankar\anaconda3\lib\site-packages\stable_baselines3\common\vec_env\patch_gym.py:49: UserWarning: You provided an OpenAI Gym environment. We strongly recommend transitioning to Gymnasium environments. Stable-Baselines3 is automatically wrapping your environments in a compatibility layer, which could potentially cause issues.
  warnings.warn(


In [8]:
# PPO?? Proximal Policy Optimization

In [9]:
model.learn(total_timesteps = 20000)


-----------------------------
| time/              |      |
|    fps             | 1068 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 692         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008388028 |
|    clip_fraction        | 0.0934      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.686      |
|    explained_variance   | -0.0134     |
|    learning_rate        | 0.0003      |
|    loss                 | 8.25        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0144     |
|    value_loss           | 53.4        |
-----------------------------------------
----------------------------------

In [10]:
PPO_path = os.path.abspath(os.path.join('Training', 'Saved Models', 'PPO_model'))

In [11]:
print("Saving model to:", PPO_path)
model.save(PPO_path)


Saving model to: c:\Users\Aditya Bankar\OneDrive\Desktop\RL\Training\Saved Models\PPO_model


In [12]:
del model

In [13]:
model = PPO.load(PPO_path,env = env)

In [14]:
evaluate_policy(model,env,n_eval_episodes = 10,render = True)

c:\Users\Aditya Bankar\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(
c:\Users\Aditya Bankar\anaconda3\lib\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:243: UserWarning: You tried to call render() but no `render_mode` was passed to the env constructor.
  warnings.warn("You tried to call render() but no `render_mode` was passed to the env constructor.")


(200.0, 0.0)

In [15]:
episodes = 5
for episode in range(1, episodes + 1):
    obs = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render(mode='ansi')  # Render the environment without specifying a render mode
        action,_ = model.predict(obs)
        step_result = env.step(action)
        obs, reward, done, info = step_result[:4]
        score += reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

c:\Users\Aditya Bankar\anaconda3\lib\site-packages\stable_baselines3\common\vec_env\base_vec_env.py:233: UserWarning: Starting from gymnasium v0.26, render modes are determined during the initialization of the environment.
                We allow to pass a mode argument to maintain a backwards compatible VecEnv API, but the mode (ansi)
                has to be the same as the environment render mode (None) which is not the case.
  warnings.warn(


Episode:1 Score:[200.]
Episode:2 Score:[200.]
Episode:3 Score:[200.]
Episode:4 Score:[104.]
Episode:5 Score:[200.]


In [16]:
training_log_path = os.path.join(log_path,'PPO_2')

In [17]:
training_log_path

'Training\\Logs\\PPO_2'

In [21]:
# !tensorboard --logdir={training_log_path}

In [22]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [23]:
save_path = os.path.join('Training','Saved Models')

In [24]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200,verbose=1)
eval_callback = EvalCallback(env,
                             callback_on_new_best=stop_callback,
                             eval_freq=10000,
                             best_model_save_path=save_path,
                             verbose=1)

In [25]:
model = PPO('MlpPolicy',env,verbose=1,tensorboard_log=log_path)

Using cpu device


In [26]:
model.learn(total_timesteps=20000,callback = eval_callback)

Logging to Training\Logs\PPO_1
-----------------------------
| time/              |      |
|    fps             | 1287 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 557        |
|    iterations           | 2          |
|    time_elapsed         | 7          |
|    total_timesteps      | 4096       |
| train/                  |            |
|    approx_kl            | 0.00843246 |
|    clip_fraction        | 0.0996     |
|    clip_range           | 0.2        |
|    entropy_loss         | -0.686     |
|    explained_variance   | -0.0018    |
|    learning_rate        | 0.0003     |
|    loss                 | 6.26       |
|    n_updates            | 10         |
|    policy_gradient_loss | -0.0157    |
|    value_loss           | 51.8       |
----------------------------------------
---------------------

c:\Users\Aditya Bankar\anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=123.40 +/- 39.06
Episode length: 123.40 +/- 39.06
-----------------------------------------
| eval/                   |             |
|    mean_ep_length       | 123         |
|    mean_reward          | 123         |
| time/                   |             |
|    total_timesteps      | 10000       |
| train/                  |             |
|    approx_kl            | 0.006370618 |
|    clip_fraction        | 0.0455      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.617      |
|    explained_variance   | 0.2         |
|    learning_rate        | 0.0003      |
|    loss                 | 35.5        |
|    n_updates            | 40          |
|    policy_gradient_loss | -0.0131     |
|    value_loss           | 71          |
-----------------------------------------
New best mean reward!
------------------------------
| time/              |       |
|    fps             | 561   |
|    iterations      | 5     |
|    ti

In [29]:
net_arch = [dict(pi = [128,128,128,128], vf = [128,128,128,128])]

In [30]:
model = PPO('MlpPolicy',env,verbose = 1,tensorboard_log = log_path,policy_kwargs={'net_arch':net_arch})

Using cpu device


c:\Users\Aditya Bankar\anaconda3\lib\site-packages\stable_baselines3\common\policies.py:484: UserWarning: As shared layers in the mlp_extractor are removed since SB3 v1.8.0, you should now pass directly a dictionary and not a list (net_arch=dict(pi=..., vf=...) instead of net_arch=[dict(pi=..., vf=...)])
  warnings.warn(


In [31]:
from stable_baselines3 import DQN

In [32]:
model = DQN('MlpPolicy',env,verbose = 1,tensorboard_log = log_path)

Using cpu device
